In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller


df = pd.read_csv('../datasets/clean_renewables_1985_2021.csv')

global_data = df[df['Entity'] == 'World'].copy()


global_data['Year'] = pd.PeriodIndex(global_data['Year'].astype(str), freq='Y')
global_data = global_data.sort_values('Year')


ts_data = global_data.set_index('Year')['Renewables (% electricity)']


plt.figure(figsize=(24, 6))
plt.plot(ts_data.index.astype(str), ts_data.values, marker='o', linestyle='-')
plt.title('Global Renewable Electricity Percentage (1985-2021)')
plt.xlabel('Year')
plt.ylabel('Renewable Percentage of Global Electricity')
plt.grid(True, alpha=0.3)
plt.show()

: 

In [ ]:

result = adfuller(ts_data)
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')


ts_data_diff = ts_data.diff().dropna()

if result[1] <= 0.05:
    d_value = 0
else:
    d_value = 1


plt.figure(figsize=(24, 6))
plt.plot(ts_data_diff.index.astype(str), ts_data_diff.values, marker='o')
plt.title('Differenced Renewable Electricity Percentage')
plt.xlabel('Year')
plt.ylabel('Yearly Change in Percentage')
plt.grid(True, alpha=0.3)
plt.show()


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(ts_data_diff, ax=ax1)
plot_pacf(ts_data_diff, ax=ax2)
plt.tight_layout()
plt.show()

In [ ]:
p, d, q = 1, d_value, 1


model = ARIMA(ts_data, order=(p, d, q))
model_fit = model.fit()
print(model_fit.summary())


pred = model_fit.predict(start=1, end=len(ts_data)-1)


plt.figure(figsize=(24, 6))


plt.plot(ts_data.index.astype(str), ts_data.values, 'o-', label='Actual', color='blue')
plt.plot(ts_data.index[1:].astype(str), pred.values, 'x--', label='Predicted', color='orange')

plt.title('Model Fit: Actual vs Predicted')
plt.xlabel('Year')
plt.ylabel('Renewable Percentage of Global Electricity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
forecast_horizon = 30
forecast = model_fit.forecast(steps=forecast_horizon)


last_year = int(ts_data.index[-1].year)
future_years = range(last_year + 1, last_year + forecast_horizon + 1)


forecast_df = pd.DataFrame({
    'Year': future_years,
    'Forecast': forecast
})


if any(forecast_df['Forecast'] >= 50):
    forecast_50_pct = forecast_df[forecast_df['Forecast'] >= 50]['Year'].iloc[0]
    print(f"Projected year to reach 50% renewable electricity: {forecast_50_pct}")
else:
    forecast_50_pct = None
    print("50% not reached within forecast horizon (by 2051)")

plt.figure(figsize=(24, 7))
plt.plot(ts_data.index.astype(str), ts_data.values, 'o-', label='Historical Data', color='blue')
plt.plot(forecast_df['Year'].astype(str), forecast_df['Forecast'], 'x--', label='ARIMA Forecast', color='green')
plt.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='50% Target')

if forecast_50_pct:
    plt.axvline(x=str(forecast_50_pct), color='purple', linestyle=':', alpha=0.7)
    plt.annotate(f'50% Target: {forecast_50_pct}',
                xy=(str(forecast_50_pct), 50),
                xytext=(0, 30),
                textcoords='offset points',
                arrowprops=dict(arrowstyle='->'))

plt.title('Global Renewable Electricity Percentage Projection', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Renewable Percentage of Global Electricity', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nForecasted values for key milestone years:")
milestone_years = [last_year + 5, last_year + 10, last_year + 20, last_year + 30]
for year in milestone_years:
    forecast_value = forecast_df[forecast_df['Year'] == year]['Forecast'].iloc[0]
    print(f"Year {year}: {forecast_value:.2f}%")

In [ ]:
base_projection = model_fit.forecast(steps=30)

accelerated_projection = []
last_value = ts_data.iloc[-1]

for i in range(len(base_projection)):

    if i == 0:
        base_change = base_projection[i] - last_value
    else:
        base_change = base_projection[i] - base_projection[i-1]


    accelerated_change = base_change * 1.5

    if i == 0:
        accelerated_projection.append(last_value + accelerated_change)
    else:
        accelerated_projection.append(accelerated_projection[i-1] + accelerated_change)

In [ ]:
last_year = ts_data.index[-1].year
last_value = ts_data.iloc[-1]

future_years = range(last_year + 1, last_year + forecast_horizon + 1)


plt.figure(figsize=(14,8))

historical_years = [period.year for period in ts_data.index]

plt.plot(historical_years, ts_data.values, 'o-', color='blue', label='Historical Data')

plt.plot(future_years, base_projection, 'x--', color='green', label='Base Projection')

plt.plot(future_years, accelerated_projection, 'x--', color='red', label='Accelerated Projection 50%')

for target in [50, 70]:
    if any(base_projection >= target):
        target_year_index = np.where(base_projection >= target)[0][0]
        target_year = future_years[target_year_index]
        plt.annotate(f"Base: {target}% in {target_year}",
                     xy=(target_year, target),
                     xytext=(target_year-5, target+5),
                     arrowprops=dict(facecolor='red', shrink=0.05, width=1.5),
                     fontsize=10)

for target in [50, 70]:
    if any(np.array(accelerated_projection) >= target):
        target_year_index = np.where(np.array(accelerated_projection) >= target)[0][0]
        target_year = future_years[target_year_index]
        plt.annotate(f"Accelerated: {target}% in {target_year}",
                     xy=(target_year, target),
                     xytext=(target_year-5, target-7),
                     arrowprops=dict(facecolor='green', shrink=0.05, width=1.5),
                     fontsize=10)

plt.title('Renewable Energy Projections vs. Climate Targets', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Renewable Energy (% of Global Electricity)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)


plt.ylim(0, 100)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.legend(loc='upper left', fontsize=12)


plt.tight_layout()
plt.show()
